In [1]:
BRANCH = 'main'

In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell

# install NeMo
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[nlp]

In [2]:
# If you're not using Colab, you might need to upgrade jupyter notebook to avoid the following error:
# 'ImportError: IProgress not found. Please update jupyter and ipywidgets.'

! pip install ipywidgets
! jupyter nbextension enable --py widgetsnbextension

# Please restart the kernel after running this cell

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [ ]:
from nemo.utils.exp_manager import exp_manager
from nemo.collections import nlp as nemo_nlp
from nemo.collections import asr as nemo_asr

import os
import wget
import torch
import pytorch_lightning as pl
from omegaconf import OmegaConf

# Task Description
Automatic Speech Recognition (ASR) systems typically generate text with no punctuation and capitalization of the words.
This tutorial explains how to implement a model in NeMo that will predict punctuation and capitalization using both text and audio for each word in a sentence to make ASR output more readable and to boost performance of the named entity recognition, machine translation or text-to-speech models.
We'll show how to train a model for this task using a pre-trained BERT and Conformer models.
For every word in our training dataset we’re going to predict:

- punctuation mark that should follow the word and
- whether the word should be capitalized

# Dataset
This model can work with any dataset as long as it follows the format specified below.
The training and evaluation data is divided into *3 files: text.txt, labels.txt and audio.txt*.
Each line of the **text.txt** file contains text sequences, where words are separated with spaces: [WORD] [SPACE] [WORD] [SPACE] [WORD], for example:



```
when is the next flight to new york
the next flight is ...
...
```



The **labels.txt** file contains corresponding labels for each word in text.txt, the labels are separated with spaces. Each label in labels.txt file consists of 2 symbols:

- the first symbol of the label indicates what punctuation mark should follow the word (where O means no punctuation needed);
- the second symbol determines if a word needs to be capitalized or not (where U indicates that the word should be upper cased, and O - no capitalization needed.)

In this tutorial, we are considering only commas, periods, and question marks the rest punctuation marks were removed. To use more punctuation marks, update the dataset to include desired labels, no changes to the model needed.

Each line of the **labels.txt** should follow the format:
[LABEL] [SPACE] [LABEL] [SPACE] [LABEL] (for labels.txt).
For example, labels for the above text.txt file should be:



```
OU OO OO OO OO OO OU ?U
OU OO OO OO ...
...
```



The complete list of all possible labels for this task used in this tutorial is: `OO, ,O, .O, ?O, OU, ,U, .U, ?U.`


The **audio.txt** file contains corresponding audio's filepath for each sample in text.txt

Each line of the **audio.txt** should follow the format:
/Path/to/audio/file.wav

For example, filepaths for the above text.txt should be:



```
/fisher/1.wav
/fisher/2.wav
...
```



## Download and preprocess the data

In this notebook we are going to use a subset of [LibriTTS](https://www.openslr.org/60/). This script will download and preprocess the LibriTTS data [NeMo/examples/nlp/token_classification/get_tatoeba_data.py](https://github.com/NVIDIA/NeMo/blob/stable/examples/nlp/token_classification/data/get_libritts_data.py).

In [ ]:
DATA_DIR = 'PATH_TO_A_DIRECTORY_WHERE_DATA_FROM_THIS_TUTORIAL_IS_STORED'
WORK_DIR = 'PATH_TO_A_DIRECTORY_WHERE_SCRIPTS_FOR_THIS_TUTORIAL_ARE_SAVED'
MODEL_CONFIG = "punctuation_capitalization_lexical_audio_config.yaml"

# model parameters
TOKENS_IN_BATCH = 1024
MAX_SEQ_LENGTH = 64
LEARNING_RATE = 0.00002
NUM_SAMPLES = 10000

In [ ]:
## download get_tatoeba_data.py script to download and preprocess the Tatoeba data
os.makedirs(WORK_DIR, exist_ok=True)
if not os.path.exists(WORK_DIR + '/get_libritts_data.py'):
    print('Downloading get_libritts_data.py...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/token_classification/data/get_libritts_data.py', WORK_DIR)
else:
    print ('get_libritts_data.py already exists')

In [ ]:
# download and preprocess the data
# we will use dev_clean and dev_other subsets
# --clean flag deletes raw data
!python $WORK_DIR/get_libritts_data.py --data_dir $DATA_DIR --clean --data_set dev_clean,dev_other

after execution of the above cell, your data folder will contain the following 3 files needed for training (raw LibriTTS data could be present if `--clean` was not used):
- labels_dev.txt
- text_dev.txt
- audio_dev.txt


In [ ]:
!ls -l $DATA_DIR

In [ ]:
# let's take a look at the data 
print('Text:')
!head -n 5 $DATA_DIR/text_dev.txt

print('\nLabels:')
!head -n 5 $DATA_DIR/labels_dev.txt

print('\nFilepaths: ')
!head -n 5 $DATA_DIR/audio_dev.txt

As you see, `get_libritts_data.py` script provides not only downloads LibriTTS but also creates labels. If you wish to preprocess your own data, use [examples/nlp/token_classification/data/prepare_data_for_punctuation_capitalization.py](https://github.com/NVIDIA/NeMo/blob/main/examples/nlp/token_classification/data/prepare_data_for_punctuation_capitalization.py) script:

In [ ]:
cwd = os.getcwd()
NEMO_ROOT = "~/NeMo"
!python $NEMO_ROOT/examples/nlp/token_classification/data/prepare_data_for_punctuation_capitalization.py \
    --source_file $DATA_DIR/text_train.txt \
    --output_dir $DATA_DIR/my_train_preprocessed

In [ ]:
!ls $DATA_DIR/my_train_preprocessed -l

# Tarred dataset

If your dataset is too large to be stored in memory, you can use tarred dataset. A tarred dataset is a collection of tarred files which contain batches ready for passing into a model.

All tar files will contain identical number of batches, so if number of batches in the dataset is not evenly divisible by parameter `--num_batches_per_tar_file` value, then up to `--num_batches_per_tar_file - 1` batches may be lost.

In [ ]:
# Number of lines in text and labels files
!wc -l $DATA_DIR/text_train.txt
!wc -l $DATA_DIR/labels_train.txt

In [ ]:
NEMO_ROOT = "~/NeMo"
!python $NEMO_ROOT/examples/nlp/token_classification/data/create_punctuation_capitalization_tarred_dataset.py \
    --text $DATA_DIR/text_dev.txt \
    --labels $DATA_DIR/labels_dev.txt \
    --output_dir $DATA_DIR/train_tarred \
    --num_batches_per_tarfile 20 \
    --tokens_in_batch 1024 \
    --lines_per_dataset_fragment 4000 \
    --tokenizer_name bert-base-uncased \
    --n_jobs 2 \
    --use_audio \
    --sample_rate 16000

In [ ]:
!ls $DATA_DIR/train_tarred -l

In [ ]:
!ls $DATA_DIR/train_tarred/*.tar | wc -l  # number of tar files

In [ ]:
!ls $DATA_DIR/train_tarred/ | grep -v '.tar'  # all not tar files

If you wish to use tarred dataset, then you need to 
- pass metadata JSON file in config parameter `model.train_ds.tar_metadata_file`,
- set `model.train_ds.use_tarred_dataset=true`.

# Model Configuration

In the Punctuation and Capitalization Model, we are jointly training two token-level classifiers on top of the fusion of pretrained [BERT](https://arxiv.org/pdf/1810.04805.pdf) and encoder of ASR model models: 
- one classifier to predict punctuation and
- the other one - capitalization.

The model is defined in a config file which declares multiple important sections. They are:
- **model**: All arguments that are related to the Model - language model, token classifiers, optimizer and schedulers, dataset and any other related information

- **trainer**: Any argument to be passed to PyTorch Lightning

See [docs](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/main/nlp/punctuation_and_capitalization.html#training-punctuation-and-capitalization-model) for full config description.

In [ ]:
# download the model's configuration file 
config_dir = WORK_DIR + '/configs/'
os.makedirs(config_dir, exist_ok=True)
if not os.path.exists(config_dir + MODEL_CONFIG):
    print('Downloading config file...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/token_classification/conf/' + MODEL_CONFIG, config_dir)
else:
    print ('config file already exists')

In [ ]:
# this line will print the entire config of the model
config_path = f'{WORK_DIR}/configs/{MODEL_CONFIG}'
print(config_path)
config = OmegaConf.load(config_path)
print(OmegaConf.to_yaml(config))

# Setting up Data within the config

Among other things, the config file contains dictionaries called `common_dataset_parameters`, `train_ds` and `validation_ds`. These are configurations used to setup the Dataset and DataLoaders of the corresponding config.

Specify paths directories with train and dev datasets in parameters `train_ds.ds_item` and `validation_ds.ds_item`.

I you want to use multiple datasets for evaluation, specify paths to the directory(ies) with evaluation file(s) in the following way:

`model.validation_ds.ds_item=[PATH_TO_DEV1,PATH_TO_DEV2]` (Note no space between the paths and square brackets).

Also notice that some configs, including `model.train_ds.ds_item`, have `???` in place of values, this values are required to be specified by the user.

Let's now add the data directory path to the config.

In [ ]:
# in this tutorial train and dev data is located in the same folder
config.model.train_ds.ds_item = DATA_DIR
config.model.validation_ds.ds_item=DATA_DIR
del config.model.test_ds  # We do not have test data, only train and dev

# Building the PyTorch Lightning Trainer

NeMo models are primarily PyTorch Lightning modules - and therefore are entirely compatible with the PyTorch Lightning ecosystem!

Let's first instantiate a Trainer object!

In [ ]:
print("Trainer config - \n")
print(OmegaConf.to_yaml(config.trainer))

In [ ]:
# lets modify some trainer configs
# checks if we have GPU available and uses it
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
config.trainer.devices = 1
config.trainer.accelerator = accelerator
config.trainer.precision = 16 if torch.cuda.is_available() else 32

# For mixed precision training, use precision=16 and amp_level=O1

# Reduces maximum number of epochs to 1 for a quick training
config.trainer.max_epochs = 1

# Remove distributed training flags
config.trainer.strategy = None

trainer = pl.Trainer(**config.trainer)

# Setting up a NeMo Experiment¶

NeMo has an experiment manager that handles logging and checkpointing for us, so let's use it!

In [ ]:
exp_dir = exp_manager(trainer, config.get("exp_manager", None))

# the exp_dir provides a path to the current experiment for easy access
exp_dir = str(exp_dir)
exp_dir

# Model Training

Before initializing the model, we might want to modify some of the model configs. For example, we might want to modify the pretrained BERT model and Conformer model.

In [ ]:
# complete list of supported BERT-like models
print(nemo_nlp.modules.get_pretrained_lm_models_list())

PRETRAINED_BERT_MODEL = "bert-base-uncased"

print(nemo_asr.models.ASRModel.list_available_models())

PRETRAINED_ASR_MODEL = "stt_en_conformer_ctc_small"

In [ ]:
# add the specified above model parameters to the config
config.model.language_model.pretrained_model_name = PRETRAINED_BERT_MODEL
config.model.train_ds.tokens_in_batch = TOKENS_IN_BATCH
config.model.validation_ds.tokens_in_batch = TOKENS_IN_BATCH
config.model.optim.lr = LEARNING_RATE
config.model.train_ds.num_samples = NUM_SAMPLES
config.model.validation_ds.num_samples = NUM_SAMPLES
config.model.audio_encoder.pretrained_model = PRETRAINED_ASR_MODEL

Now, we are ready to initialize our model. During the model initialization call, the dataset and data loaders we'll be prepared for training and evaluation.
Also, the pretrained BERT model will be downloaded, note it can take up to a few minutes depending on the size of the chosen BERT model.

In [ ]:
# initialize the model
# during this stage, the dataset and data loaders we'll be prepared for training and evaluation
model = nemo_nlp.models.PunctuationCapitalizationModel(cfg=config.model, trainer=trainer)

## Monitoring training progress
Optionally, you can create a Tensorboard visualization to monitor training progress.

In [ ]:
try:
  from google import colab
  COLAB_ENV = True
except (ImportError, ModuleNotFoundError):
  COLAB_ENV = False

# Load the TensorBoard notebook extension
if COLAB_ENV:
  %load_ext tensorboard
  %tensorboard --logdir {exp_dir}
else:
  print("To use tensorboard, please use this notebook in a Google Colab environment.")

In [ ]:
# start the training
trainer.fit(model)

# Training using tarred dataset


In [ ]:
config = OmegaConf.load(config_path)
config.model.train_ds.ds_item = f'{DATA_DIR}/train_tarred'
config.model.train_ds.use_tarred_dataset = True
# Only metadata file name is required if `use_tarred_dataset=true`.
config.model.train_ds.tar_metadata_file = 'metadata.punctuation_capitalization.tokens1024.max_seq_length512.bert-base-uncased.json'
config.model.validation_ds.ds_item = DATA_DIR
del config.model.test_ds  # We do not have test data, only train and dev

# Trainer
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
config.trainer.devices = 1
config.trainer.accelerator = accelerator
config.trainer.precision = 16 if torch.cuda.is_available() else 32
config.trainer.max_epochs = 1
config.trainer.strategy = None

# Exp manager
config.exp_manager.explicit_log_dir = 'tarred_experiment'

config.model.language_model.pretrained_model_name = PRETRAINED_BERT_MODEL
config.model.validation_ds.tokens_in_batch = TOKENS_IN_BATCH
config.model.optim.lr = LEARNING_RATE
config.model.validation_ds.num_samples = NUM_SAMPLES

In [ ]:
trainer = pl.Trainer(**config.trainer)
exp_dir = exp_manager(trainer, config.get("exp_manager", None))
model = nemo_nlp.models.PunctuationCapitalizationModel(cfg=config.model, trainer=trainer)
trainer.fit(model)

# Inference using a pretrained model

For inference you can use same script as PunctuationCapitalizationModel uses, just add **--use_audio** and **--audio_file** parameters

## Training Script

If you have NeMo installed locally, you can also train the model with [nlp/token_classification/punctuation_capitalization_lexical_audio_train_evaluate.py](https://github.com/NVIDIA/NeMo/blob/main/examples/nlp/token_classification/punctuation_capitalization_lexical_audio_train_evaluate.py).

To run training script, use:

`python punctuation_capitalization_lexical_audio_train_evaluate.py model.train_ds.ds_item=PATH_TO_TRAIN_DATA_DIR`

Set NUM_SAMPLES=-1 and consider including other datasets to improve the performance of the model.

# Finetuning model with your data

When we were training the model from scratch, the datasets were prepared for training during the model initialization. When we are using a pretrained Punctuation and Capitalization model, before training, we need to setup training and evaluation data.

In [ ]:
# let's reload our pretrained model
pretrained_model = nemo_nlp.models.PunctuationCapitalizationLexicalAudioModel.from_pretrained('PLACEHOLDER')

# setup train and validation Pytorch DataLoaders
pretrained_model.update_config_after_restoring_from_checkpoint(
    train_ds={
        'ds_item': DATA_DIR,
        'text_file': 'text_train.txt',
        'labels_file': 'labels_train.txt',
        'tokens_in_batch': 1024,
    },
    validation_ds={
        'ds_item': DATA_DIR,
        'text_file': 'text_dev.txt',
        'labels_file': 'labels_dev.txt',
        'tokens_in_batch': 1024,
    },
)

# and now we can create a PyTorch Lightning trainer and call `fit` again
# for this tutorial we are setting fast_dev_run to True, and the trainer will run 1 training batch and 1 validation batch
# for actual model training, disable the flag
fast_dev_run = True
trainer = pl.Trainer(devices=1, accelerator='gpu', fast_dev_run=fast_dev_run)
pretrained_model.set_trainer(trainer)
pretrained_model.setup_training_data()
pretrained_model.setup_validation_data()
trainer.fit(pretrained_model)

In this notebook we are going to use a subset of [LibriTTS](https://www.openslr.org/60/). This script will download and preprocess the LibriTTS data [NeMo/examples/nlp/token_classification/get_tatoeba_data.py](https://github.com/NVIDIA/NeMo/blob/stable/examples/nlp/token_classification/data/get_libritts_data.py).

In [19]:
DATA_DIR = 'PATH_TO_A_DIRECTORY_WHERE_DATA_FROM_THIS_TUTORIAL_IS_STORED'
WORK_DIR = 'PATH_TO_A_DIRECTORY_WHERE_SCRIPTS_FOR_THIS_TUTORIAL_ARE_SAVED'
MODEL_CONFIG = "punctuation_capitalization_lexical_audio_config.yaml"

# model parameters
TOKENS_IN_BATCH = 1024
MAX_SEQ_LENGTH = 64
LEARNING_RATE = 0.00002
NUM_SAMPLES = 10000

In [6]:
## download get_tatoeba_data.py script to download and preprocess the Tatoeba data
os.makedirs(WORK_DIR, exist_ok=True)
if not os.path.exists(WORK_DIR + '/get_libritts_data.py'):
    print('Downloading get_libritts_data.py...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/token_classification/data/get_libritts_data.py', WORK_DIR)
else:
    print ('get_libritts_data.py already exists')

HTTPError: HTTP Error 404: Not Found

In [8]:
# download and preprocess the data
# we will use dev_clean and dev_other subsets
# --clean flag deletes raw data
!python $WORK_DIR/get_libritts_data.py --data_dir $DATA_DIR --clean --data_set dev_clean,dev_other

python: can't open file '/home/mnovikov/PycharmProjects/NeMo/tutorials/nlp/PATH_TO_A_DIRECTORY_WHERE_SCRIPTS_FOR_THIS_TUTORIAL_ARE_SAVED/get_libritts_data.py': [Errno 2] No such file or directory


after execution of the above cell, your data folder will contain the following 3 files needed for training (raw LibriTTS data could be present if `--clean` was not used):
- labels_dev.txt
- text_dev.txt
- audio_dev.txt


In [9]:
!ls -l $DATA_DIR

ls: cannot access 'PATH_TO_A_DIRECTORY_WHERE_DATA_FROM_THIS_TUTORIAL_IS_STORED': No such file or directory


In [11]:
# let's take a look at the data 
print('Text:')
!head -n 5 $DATA_DIR/text_dev.txt

print('\nLabels:')
!head -n 5 $DATA_DIR/labels_dev.txt

print('\nFilepaths: ')
!head -n 5 $DATA_DIR/audio_dev.txt

Text:
head: cannot open 'PATH_TO_A_DIRECTORY_WHERE_DATA_FROM_THIS_TUTORIAL_IS_STORED/text_dev.txt' for reading: No such file or directory

Labels:
head: cannot open 'PATH_TO_A_DIRECTORY_WHERE_DATA_FROM_THIS_TUTORIAL_IS_STORED/labels_dev.txt' for reading: No such file or directory

Filepaths: 
head: cannot open 'PATH_TO_A_DIRECTORY_WHERE_DATA_FROM_THIS_TUTORIAL_IS_STORED/audio_dev.txt' for reading: No such file or directory


As you see, `get_libritts_data.py` script provides not only downloads LibriTTS but also creates labels. If you wish to preprocess your own data, use [examples/nlp/token_classification/data/prepare_data_for_punctuation_capitalization.py](https://github.com/NVIDIA/NeMo/blob/main/examples/nlp/token_classification/data/prepare_data_for_punctuation_capitalization.py) script:

In [12]:
cwd = os.getcwd()
NEMO_ROOT = "~/NeMo"
!python $NEMO_ROOT/examples/nlp/token_classification/data/prepare_data_for_punctuation_capitalization.py \
    --source_file $DATA_DIR/text_train.txt \
    --output_dir $DATA_DIR/my_train_preprocessed

python: can't open file '/home/mnovikov/NeMo/examples/nlp/token_classification/data/prepare_data_for_punctuation_capitalization.py': [Errno 2] No such file or directory


In [13]:
!ls $DATA_DIR/my_train_preprocessed -l

ls: cannot access 'PATH_TO_A_DIRECTORY_WHERE_DATA_FROM_THIS_TUTORIAL_IS_STORED/my_train_preprocessed': No such file or directory


# Tarred dataset

If your dataset is too large to be stored in memory, you can use tarred dataset. A tarred dataset is a collection of tarred files which contain batches ready for passing into a model.

All tar files will contain identical number of batches, so if number of batches in the dataset is not evenly divisible by parameter `--num_batches_per_tar_file` value, then up to `--num_batches_per_tar_file - 1` batches may be lost.

In [14]:
# Number of lines in text and labels files
!wc -l $DATA_DIR/text_train.txt
!wc -l $DATA_DIR/labels_train.txt

wc: PATH_TO_A_DIRECTORY_WHERE_DATA_FROM_THIS_TUTORIAL_IS_STORED/text_train.txt: No such file or directory
wc: PATH_TO_A_DIRECTORY_WHERE_DATA_FROM_THIS_TUTORIAL_IS_STORED/labels_train.txt: No such file or directory


In [15]:
NEMO_ROOT = "~/NeMo"
!python $NEMO_ROOT/examples/nlp/token_classification/data/create_punctuation_capitalization_tarred_dataset.py \
    --text $DATA_DIR/text_dev.txt \
    --labels $DATA_DIR/labels_dev.txt \
    --output_dir $DATA_DIR/train_tarred \
    --num_batches_per_tarfile 20 \
    --tokens_in_batch 1024 \
    --lines_per_dataset_fragment 4000 \
    --tokenizer_name bert-base-uncased \
    --n_jobs 2 \
    --use_audio \
    --sample_rate 16000

python: can't open file '/home/mnovikov/NeMo/examples/nlp/token_classification/data/create_punctuation_capitalization_tarred_dataset.py': [Errno 2] No such file or directory


In [16]:
!ls $DATA_DIR/train_tarred -l

ls: cannot access 'PATH_TO_A_DIRECTORY_WHERE_DATA_FROM_THIS_TUTORIAL_IS_STORED/train_tarred': No such file or directory


In [17]:
!ls $DATA_DIR/train_tarred/*.tar | wc -l  # number of tar files

ls: cannot access 'PATH_TO_A_DIRECTORY_WHERE_DATA_FROM_THIS_TUTORIAL_IS_STORED/train_tarred/*.tar': No such file or directory
0


In [18]:
!ls $DATA_DIR/train_tarred/ | grep -v '.tar'  # all not tar files

ls: cannot access 'PATH_TO_A_DIRECTORY_WHERE_DATA_FROM_THIS_TUTORIAL_IS_STORED/train_tarred/': No such file or directory


If you wish to use tarred dataset, then you need to 
- pass metadata JSON file in config parameter `model.train_ds.tar_metadata_file`,
- set `model.train_ds.use_tarred_dataset=true`.

# Model Configuration

In the Punctuation and Capitalization Model, we are jointly training two token-level classifiers on top of the fusion of pretrained [BERT](https://arxiv.org/pdf/1810.04805.pdf) and encoder of ASR model models: 
- one classifier to predict punctuation and
- the other one - capitalization.

The model is defined in a config file which declares multiple important sections. They are:
- **model**: All arguments that are related to the Model - language model, token classifiers, optimizer and schedulers, dataset and any other related information

- **trainer**: Any argument to be passed to PyTorch Lightning

See [docs](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/main/nlp/punctuation_and_capitalization.html#training-punctuation-and-capitalization-model) for full config description.

In [20]:
# download the model's configuration file 
config_dir = WORK_DIR + '/configs/'
os.makedirs(config_dir, exist_ok=True)
if not os.path.exists(config_dir + MODEL_CONFIG):
    print('Downloading config file...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/token_classification/conf/' + MODEL_CONFIG, config_dir)
else:
    print ('config file already exists')

100% [................................................................................] 9172 / 9172

In [21]:
# this line will print the entire config of the model
config_path = f'{WORK_DIR}/configs/{MODEL_CONFIG}'
print(config_path)
config = OmegaConf.load(config_path)
print(OmegaConf.to_yaml(config))

PATH_TO_A_DIRECTORY_WHERE_SCRIPTS_FOR_THIS_TUTORIAL_ARE_SAVED/configs/punctuation_capitalization_lexical_audio_config.yaml
pretrained_model: null
trainer:
  devices: -1
  num_nodes: 1
  max_epochs: 5
  max_steps: -1
  accumulate_grad_batches: 1
  gradient_clip_val: 0.0
  precision: 32
  accelerator: gpu
  strategy: ddp
  enable_checkpointing: false
  logger: false
  val_check_interval: 1.0
  resume_from_checkpoint: null
  log_every_n_steps: 50
exp_manager:
  exp_dir: null
  name: Punctuation_and_Capitalization_Lexical_Audio
  create_tensorboard_logger: true
  create_checkpoint_callback: true
  checkpoint_callback_params:
    save_top_k: 3
    monitor: val_loss
    mode: min
    save_best_model: true
model:
  audio_encoder:
    pretrained_model: stt_en_conformer_ctc_medium
    freeze:
      is_enabled: false
      d_model: 256
      d_ff: 1024
      num_layers: 4
    adapter:
      enable: false
      config:
        in_features: -1
        dim: 128
        activation: swish
    fusion:

# Setting up Data within the config

Among other things, the config file contains dictionaries called `common_dataset_parameters`, `train_ds` and `validation_ds`. These are configurations used to setup the Dataset and DataLoaders of the corresponding config.

Specify paths directories with train and dev datasets in parameters `train_ds.ds_item` and `validation_ds.ds_item`.

I you want to use multiple datasets for evaluation, specify paths to the directory(ies) with evaluation file(s) in the following way:

`model.validation_ds.ds_item=[PATH_TO_DEV1,PATH_TO_DEV2]` (Note no space between the paths and square brackets).

Also notice that some configs, including `model.train_ds.ds_item`, have `???` in place of values, this values are required to be specified by the user.

Let's now add the data directory path to the config.

In [22]:
# in this tutorial train and dev data is located in the same folder
config.model.train_ds.ds_item = DATA_DIR
config.model.validation_ds.ds_item=DATA_DIR
del config.model.test_ds  # We do not have test data, only train and dev

# Building the PyTorch Lightning Trainer

NeMo models are primarily PyTorch Lightning modules - and therefore are entirely compatible with the PyTorch Lightning ecosystem!

Let's first instantiate a Trainer object!

In [23]:
print("Trainer config - \n")
print(OmegaConf.to_yaml(config.trainer))

Trainer config - 

devices: -1
num_nodes: 1
max_epochs: 5
max_steps: -1
accumulate_grad_batches: 1
gradient_clip_val: 0.0
precision: 32
accelerator: gpu
strategy: ddp
enable_checkpointing: false
logger: false
val_check_interval: 1.0
resume_from_checkpoint: null
log_every_n_steps: 50



In [24]:
# lets modify some trainer configs
# checks if we have GPU available and uses it
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
config.trainer.devices = 1
config.trainer.accelerator = accelerator
config.trainer.precision = 16 if torch.cuda.is_available() else 32

# For mixed precision training, use precision=16 and amp_level=O1

# Reduces maximum number of epochs to 1 for a quick training
config.trainer.max_epochs = 1

# Remove distributed training flags
config.trainer.strategy = None

trainer = pl.Trainer(**config.trainer)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


# Setting up a NeMo Experiment¶

NeMo has an experiment manager that handles logging and checkpointing for us, so let's use it!

In [25]:
exp_dir = exp_manager(trainer, config.get("exp_manager", None))

# the exp_dir provides a path to the current experiment for easy access
exp_dir = str(exp_dir)
exp_dir

[NeMo I 2022-09-26 08:55:29 exp_manager:286] Experiments will be logged at /home/mnovikov/PycharmProjects/NeMo/tutorials/nlp/nemo_experiments/Punctuation_and_Capitalization_Lexical_Audio/2022-09-26_08-55-29
[NeMo I 2022-09-26 08:55:29 exp_manager:660] TensorboardLogger has been set up


[NeMo W 2022-09-26 08:55:29 nemo_logging:349] /home/mnovikov/miniconda3/envs/NeMo/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:2274: LightningDeprecationWarning: `Trainer.weights_save_path` has been deprecated in v1.6 and will be removed in v1.8.
      rank_zero_deprecation("`Trainer.weights_save_path` has been deprecated in v1.6 and will be removed in v1.8.")
    
[NeMo W 2022-09-26 08:55:29 exp_manager:900] The checkpoint callback was told to monitor a validation value and trainer's max_steps was set to -1. Please ensure that max_steps will run for at least 1 epochs to ensure that checkpointing will not error out.


'/home/mnovikov/PycharmProjects/NeMo/tutorials/nlp/nemo_experiments/Punctuation_and_Capitalization_Lexical_Audio/2022-09-26_08-55-29'

# Model Training

Before initializing the model, we might want to modify some of the model configs. For example, we might want to modify the pretrained BERT model and Conformer model.

In [31]:
# complete list of supported BERT-like models
print(nemo_nlp.modules.get_pretrained_lm_models_list())

PRETRAINED_BERT_MODEL = "bert-base-uncased"

print(nemo_asr.models.ASRModel.list_available_models())

PRETRAINED_ASR_MODEL = "stt_en_conformer_ctc_small"

['bert-base-uncased', 'bert-large-uncased', 'bert-base-cased', 'bert-large-cased', 'bert-base-multilingual-uncased', 'bert-base-multilingual-cased', 'bert-base-chinese', 'bert-base-german-cased', 'bert-large-uncased-whole-word-masking', 'bert-large-cased-whole-word-masking', 'bert-large-uncased-whole-word-masking-finetuned-squad', 'bert-large-cased-whole-word-masking-finetuned-squad', 'bert-base-cased-finetuned-mrpc', 'bert-base-german-dbmdz-cased', 'bert-base-german-dbmdz-uncased', 'cl-tohoku/bert-base-japanese', 'cl-tohoku/bert-base-japanese-whole-word-masking', 'cl-tohoku/bert-base-japanese-char', 'cl-tohoku/bert-base-japanese-char-whole-word-masking', 'TurkuNLP/bert-base-finnish-cased-v1', 'TurkuNLP/bert-base-finnish-uncased-v1', 'wietsedv/bert-base-dutch-cased', 'distilbert-base-uncased', 'distilbert-base-uncased-distilled-squad', 'distilbert-base-cased', 'distilbert-base-cased-distilled-squad', 'distilbert-base-german-cased', 'distilbert-base-multilingual-cased', 'distilbert-base

In [33]:
# add the specified above model parameters to the config
config.model.language_model.pretrained_model_name = PRETRAINED_BERT_MODEL
config.model.train_ds.tokens_in_batch = TOKENS_IN_BATCH
config.model.validation_ds.tokens_in_batch = TOKENS_IN_BATCH
config.model.optim.lr = LEARNING_RATE
config.model.train_ds.num_samples = NUM_SAMPLES
config.model.validation_ds.num_samples = NUM_SAMPLES
config.model.audio_encoder.pretrained_model = PRETRAINED_ASR_MODEL

Now, we are ready to initialize our model. During the model initialization call, the dataset and data loaders we'll be prepared for training and evaluation.
Also, the pretrained BERT model will be downloaded, note it can take up to a few minutes depending on the size of the chosen BERT model.

In [34]:
# initialize the model
# during this stage, the dataset and data loaders we'll be prepared for training and evaluation
model = nemo_nlp.models.PunctuationCapitalizationModel(cfg=config.model, trainer=trainer)

[NeMo I 2022-09-26 08:59:36 tokenizer_utils:130] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-base-uncased, vocab_file: None, merges_files: None, special_tokens_dict: {}, and use_fast: False


Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.
[NeMo W 2022-09-26 08:59:38 modelPT:217] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.


FileNotFoundError: PATH_TO_A_DIRECTORY_WHERE_DATA_FROM_THIS_TUTORIAL_IS_STORED/text_train.txt or PATH_TO_A_DIRECTORY_WHERE_DATA_FROM_THIS_TUTORIAL_IS_STORED/labels_train.txt not found. The data should be split into 2 files: text.txt and labels.txt. Each line of the text.txt file contains text sequences, where words are separated with spaces. The labels.txt file contains corresponding labels for each word in text.txt, the labels are separated with spaces. Each line of the files should follow the format:
   [WORD] [SPACE] [WORD] [SPACE] [WORD] (for text.txt) and    [LABEL] [SPACE] [LABEL] [SPACE] [LABEL] (for labels.txt).

## Monitoring training progress
Optionally, you can create a Tensorboard visualization to monitor training progress.

In [35]:
try:
  from google import colab
  COLAB_ENV = True
except (ImportError, ModuleNotFoundError):
  COLAB_ENV = False

# Load the TensorBoard notebook extension
if COLAB_ENV:
  %load_ext tensorboard
  %tensorboard --logdir {exp_dir}
else:
  print("To use tensorboard, please use this notebook in a Google Colab environment.")

To use tensorboard, please use this notebook in a Google Colab environment.


In [ ]:
# start the training
trainer.fit(model)

# Training using tarred dataset


In [36]:
config = OmegaConf.load(config_path)
config.model.train_ds.ds_item = f'{DATA_DIR}/train_tarred'
config.model.train_ds.use_tarred_dataset = True
# Only metadata file name is required if `use_tarred_dataset=true`.
config.model.train_ds.tar_metadata_file = 'metadata.punctuation_capitalization.tokens1024.max_seq_length512.bert-base-uncased.json'
config.model.validation_ds.ds_item = DATA_DIR
del config.model.test_ds  # We do not have test data, only train and dev

# Trainer
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
config.trainer.devices = 1
config.trainer.accelerator = accelerator
config.trainer.precision = 16 if torch.cuda.is_available() else 32
config.trainer.max_epochs = 1
config.trainer.strategy = None

# Exp manager
config.exp_manager.explicit_log_dir = 'tarred_experiment'

config.model.language_model.pretrained_model_name = PRETRAINED_BERT_MODEL
config.model.validation_ds.tokens_in_batch = TOKENS_IN_BATCH
config.model.optim.lr = LEARNING_RATE
config.model.validation_ds.num_samples = NUM_SAMPLES

In [ ]:
trainer = pl.Trainer(**config.trainer)
exp_dir = exp_manager(trainer, config.get("exp_manager", None))
model = nemo_nlp.models.PunctuationCapitalizationModel(cfg=config.model, trainer=trainer)
trainer.fit(model)

# Inference using a pretrained model

For inference you can use same script as PunctuationCapitalizationModel uses, just add **--use_audio** and **--audio_file** parameters

## Training Script

If you have NeMo installed locally, you can also train the model with [nlp/token_classification/punctuation_capitalization_lexical_audio_train_evaluate.py](https://github.com/NVIDIA/NeMo/blob/main/examples/nlp/token_classification/punctuation_capitalization_lexical_audio_train_evaluate.py).

To run training script, use:

`python punctuation_capitalization_lexical_audio_train_evaluate.py model.train_ds.ds_item=PATH_TO_TRAIN_DATA_DIR`

Set NUM_SAMPLES=-1 and consider including other datasets to improve the performance of the model.

# Finetuning model with your data

When we were training the model from scratch, the datasets were prepared for training during the model initialization. When we are using a pretrained Punctuation and Capitalization model, before training, we need to setup training and evaluation data.

In [37]:
# let's reload our pretrained model
pretrained_model = nemo_nlp.models.PunctuationCapitalizationLexicalAudioModel.from_pretrained('PLACEHOLDER')

# setup train and validation Pytorch DataLoaders
pretrained_model.update_config_after_restoring_from_checkpoint(
    train_ds={
        'ds_item': DATA_DIR,
        'text_file': 'text_train.txt',
        'labels_file': 'labels_train.txt',
        'tokens_in_batch': 1024,
    },
    validation_ds={
        'ds_item': DATA_DIR,
        'text_file': 'text_dev.txt',
        'labels_file': 'labels_dev.txt',
        'tokens_in_batch': 1024,
    },
)

# and now we can create a PyTorch Lightning trainer and call `fit` again
# for this tutorial we are setting fast_dev_run to True, and the trainer will run 1 training batch and 1 validation batch
# for actual model training, disable the flag
fast_dev_run = True
trainer = pl.Trainer(devices=1, accelerator='gpu', fast_dev_run=fast_dev_run)
pretrained_model.set_trainer(trainer)
pretrained_model.setup_training_data()
pretrained_model.setup_validation_data()
trainer.fit(pretrained_model)

FileNotFoundError: Model PLACEHOLDER was not found. Check cls.list_available_models() for the list of all available models.